## 1. Import Dependencies

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

## 2. Instantiae Model

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

## 3. Encode and Calculate Sentiment

In [9]:
tokens = tokenizer.encode("Nadzmi has the worst fart! It's the stinkiest and most busuk fart ever", return_tensors='pt')

In [21]:
tokens

tensor([[  101, 12145, 75520, 10438, 10103, 43060, 12218, 10123,   106, 10197,
           112,   161, 10103, 85270, 58476, 14324, 10110, 10889, 15952, 12721,
         12218, 10123, 15765,   102]])

In [22]:
tokenizer.decode(tokens[0])

"[CLS] nadzmi has the worst fart! it's the stinkiest and most busuk fart ever [SEP]"

In [25]:
result = model(tokens)

In [39]:
result.logits

tensor([[ 4.1260,  1.5368, -0.5114, -2.6177, -1.8860]],
       grad_fn=<AddmmBackward0>)

In [42]:
torch.argmax(result.logits) + 1

tensor(1)

In [43]:
torch.argmax(result[0]) + 1

tensor(1)

In [44]:
# Try another string
tokens = tokenizer.encode("She is as pretty as the moon!", return_tensors='pt')
result = model(tokens)
torch.argmax(result.logits) + 1

tensor(5)

In [65]:
# Try another string
tokens = tokenizer.encode("It is meh", return_tensors='pt')
result = model(tokens)
torch.argmax(result.logits) + 1

tensor(3)

## 4. Collect Reviews

## 5. Load Reviews into DataFrame